# **데이터 2차 전처리**

In [1]:
import pandas as pd
import numpy as np
from functools import reduce
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
import re
from os import walk
filenames = next(walk("./data/"), (None, None, []))[2]

In [3]:
filenames

['공감수_순위_주간.csv',
 '기기별분포_주간.csv',
 '방문횟수_일간.csv',
 '방문횟수_주간.csv',
 '성연령별분포_주간.csv',
 '순방문자수_일간.csv',
 '순방문자수_주간.csv',
 '유입분석_주간.csv',
 '이웃방문현황_일간.csv',
 '이웃방문현황_주간.csv',
 '이웃증감수_일간.csv',
 '이웃증감수_주간.csv',
 '일간.csv',
 '재방문율_주간.csv',
 '조회수_순위_주간.csv',
 '조회수_일간.csv',
 '조회수_주간.csv',
 '주간.csv',
 '평균방문횟수_주간.csv',
 '평균사용시간_일간.csv',
 '평균사용시간_주간.csv']

## **주간 데이터 전처리**

In [4]:
matches = list(map(lambda x: x[0], filter(None, map(lambda x: re.findall(r'.*_주간.*', x), filenames))))
matches

['공감수_순위_주간.csv',
 '기기별분포_주간.csv',
 '방문횟수_주간.csv',
 '성연령별분포_주간.csv',
 '순방문자수_주간.csv',
 '유입분석_주간.csv',
 '이웃방문현황_주간.csv',
 '이웃증감수_주간.csv',
 '재방문율_주간.csv',
 '조회수_순위_주간.csv',
 '조회수_주간.csv',
 '평균방문횟수_주간.csv',
 '평균사용시간_주간.csv']

In [5]:
방문횟수_주간 = pd.read_csv('data/방문횟수_주간.csv')
순방문자수_주간 = pd.read_csv('data/순방문자수_주간.csv')
순방문자수_주간.columns=['기간', '순방문자수_전체', '순방문자수_피이웃', '순방문자수_서로이웃', '순방문자수_기타']
이웃방문현황_주간 = pd.read_csv('data/이웃방문현황_주간.csv')
이웃증감수_주간 = pd.read_csv('data/이웃증감수_주간.csv')
재방문율_주간 = pd.read_csv('data/재방문율_주간.csv')
평균방문횟수_주간 = pd.read_csv('data/평균방문횟수_주간.csv')
평균방문횟수_주간.columns=['기간', '평균방문횟수']
평균사용시간_주간 = pd.read_csv('data/평균사용시간_주간.csv')
평균사용시간_주간.columns=['기간', '평균사용시간']

In [6]:
dfs = [value for name, value in locals().items() if name.endswith('_주간')]

In [7]:
주간데이터 = reduce(lambda left,right: pd.merge(left, right, on=['기간']), dfs)

In [8]:
주간데이터.to_csv('data/주간.csv', index=False)

## **일간 데이터 전처리**

In [9]:
matches = list(map(lambda x: x[0], filter(None, map(lambda x: re.findall(r'.*_일간.*', x), filenames))))
matches

['방문횟수_일간.csv',
 '순방문자수_일간.csv',
 '이웃방문현황_일간.csv',
 '이웃증감수_일간.csv',
 '조회수_일간.csv',
 '평균사용시간_일간.csv']

In [10]:
방문횟수_일간 = pd.read_csv('data/방문횟수_일간.csv')
순방문자수_일간 = pd.read_csv('data/순방문자수_일간.csv')
순방문자수_일간.columns=['날짜', '요일', '순방문자수_전체', '순방문자수_피이웃', '순방문자수_서로이웃', '순방문자수_기타']
이웃방문현황_일간 = pd.read_csv('data/이웃방문현황_일간.csv')
이웃증감수_일간 = pd.read_csv('data/이웃증감수_일간.csv')
평균사용시간_일간 = pd.read_csv('data/평균사용시간_일간.csv')
평균사용시간_일간.columns=['날짜', '요일', '평균사용시간']

In [11]:
dfs = [value for name, value in locals().items() if name.endswith('_일간')]

In [12]:
일간데이터 = reduce(lambda left,right: pd.merge(left, right, on=['날짜', '요일']), dfs)

## **주간-일간 매핑**

In [22]:
주간데이터['min기간'] = 주간데이터.기간.str.slice(0, 10)
주간데이터['max기간'] = 주간데이터.기간.str.slice(11, )

In [27]:
a = 일간데이터.날짜.values
bh = 주간데이터.max기간.values
bl = 주간데이터.min기간.values

i, j = np.where((a[:, None] >= bl) & (a[:, None] <= bh))

일간데이터2 = pd.concat([
    일간데이터.loc[i, :].reset_index(drop=True),
    주간데이터.loc[j, '기간'].reset_index(drop=True)
], axis=1)

In [36]:
일간데이터2.to_csv('data/일간.csv', index=False)